In [1]:
import pandas as pd

csv_data = pd.read_csv('click_stream3.csv', names=['user_id', 'page', 'event_date', 'device', 'sex'], parse_dates=['event_date'])
csv_data

,user_id,page,event_date,device,sex
0,313593,1_home_page,2015-02-26,Desktop,Female
1,468315,1_home_page,2015-02-21,Desktop,Male
2,264005,1_home_page,2015-03-25,Desktop,Female
3,290784,1_home_page,2015-03-14,Desktop,Male
4,639104,1_home_page,2015-01-03,Desktop,Female
...,...,...,...,...,...
142077,397473,4_payment_confirmation_page,2015-01-12,Mobile,Female
142078,860829,4_payment_confirmation_page,2015-02-07,Mobile,Female
142079,371291,4_payment_confirmation_page,2015-02-17,Mobile,Female
142080,263707,4_payment_confirmation_page,2015-03-20,Mobile,Female


In [2]:
# Группирую данные по месяцам, а внутри них по переходам на страницы
funnel_data = csv_data.groupby([pd.Grouper(key='event_date', freq='M'), 'page', 'device', 'sex'])['user_id'].count().to_frame()
funnel_data['user_percent']=0.0
funnel_data.columns = ['user_count', 'user_percent']
funnel_data

user_count  \
event_date page                        device  sex                  
2015-01-31 1_home_page                 Desktop Female        7546   
                                               Male          7504   
                                       Mobile  Female        3868   
                                               Male          3682   
           2_search_page               Desktop Female        3851   
...                                                           ...   
2015-04-30 3_payment_page              Mobile  Male           148   
           4_payment_confirmation_page Desktop Female           9   
                                               Male            13   
                                       Mobile  Female          11   
                                               Male            13   

                                                       user_percent  
event_date page                        device  sex                   
2015-01-31 1_home_page                 Desktop Female           0.0  
                                               Male             0.0  
                                       Mobile  Female           0.0  
                                               Male             0.0  
           2_search_page               Desktop Female           0.0  
...                                                             ...  
2015-04-30 3_payment_page              Mobile  Male             0.0  
           4_payment_confirmation_page Desktop Female           0.0  
                                               Male             0.0  
                                       Mobile  Female           0.0  
                                               Male             0.0  

[64 rows x 2 columns]

In [3]:
# Перевожу абсолютные значения в процентные
dates_col = len(funnel_data.index.get_level_values(level=0).unique()) # кол-во диапазонов дат
page_col = len(funnel_data.index.get_level_values(level=1).unique()) # кол-во видов страниц
device_col = len(funnel_data.index.get_level_values(level=2).unique())
sex_col = len(funnel_data.index.get_level_values(level=3).unique())

for i in range(0, dates_col): # цикл по датам
    for j in range(0, page_col): # цикл по страницам внутри даты
        for k in range(0, device_col): # цикл по видам устройств
            for l in range(0, sex_col):
                pos = i*page_col*device_col*sex_col+j*device_col*sex_col+k*sex_col+l # номер текущего элемента
                base_pos = i*page_col*device_col*sex_col+k*sex_col+l # номер элемента, берущегося за 100% в данном диапазоне дат
                funnel_data.iloc[pos, 1] = round(funnel_data.iloc[pos, 0] / funnel_data.iloc[base_pos, 0]*100, 2)
    #             print(i, j, k, pos, i*j*k, base_pos, funnel_data.iloc[pos, 0], funnel_data.iloc[base_pos, 0] )

print('Полученные данные для воронок по датам и типам устройств, в абсолютном и процентном соотношении')
print(funnel_data)

Полученные данные для воронок по датам и типам устройств, в абсолютном и процентном соотношении
                                                       user_count  \
event_date page                        device  sex                  
2015-01-31 1_home_page                 Desktop Female        7546   
                                               Male          7504   
                                       Mobile  Female        3868   
                                               Male          3682   
           2_search_page               Desktop Female        3851   
...                                                           ...   
2015-04-30 3_payment_page              Mobile  Male           148   
           4_payment_confirmation_page Desktop Female           9   
                                               Male            13   
                                       Mobile  Female          11   
                                               Male            13   

      

In [4]:
# Визуализируем данные
funnel_data.sort_index(level=[0,1], ascending=False).plot.barh(y='user_percent', figsize=(16,16), legend=False, 
                                                               title='Воронка продаж по датам, страницам, типам устройств и полу клиентов');

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Глядя на график, можно сделать вывод, что с марте резко упала эффективность привлечения клиентов на мобильных устройствах, в разделе "search_page"